In [ ]:
# 公用模块

import math
import numpy as np
from typing import *

from build123d import *

In [ ]:
# 脚本参数

VISUALIZE = True
ASSEMBLE = False
EXPORT = True

if VISUALIZE:
    from ocp_vscode import *

In [ ]:
# 全局设计参数

UNIT = 19.

## 对PCB进行逆向

In [ ]:
__PCB_ORIGIN = np.array([10.6075, 19.1550])  # 本代码中使用kle中的原点为各个图形的原点。该值为该原点在pcb中的坐标
__SWITCHES_ROW1_LOCS_RAW = [  # x, y
    (20.1075, 35.7800), (39.1075, 35.7800), (58.1075, 31.0300), (77.1075, 28.6550), (96.1075, 31.0300), (115.1075, 33.4050),
]
__SWITCHES_THUMBS_LOCS_RAW = [  # x, y, r, w, h
    (86.6075, 88.6550, 0, 1, 1), (107.6075, 91.4050, -15, 1, 1), (129.8575, 95.1550, 60, 1.5, 1)
]

SWITCHES_SIZES = [
    *[(UNIT, UNIT) for _ in range(len(__SWITCHES_ROW1_LOCS_RAW) * 3)],
    *[(w * UNIT, h * UNIT) for _, _, _, w, h in __SWITCHES_THUMBS_LOCS_RAW]
]

SWITCHES_LOCS: List[Location] = []
for row in range(3):
    for x, y in __SWITCHES_ROW1_LOCS_RAW:
        SWITCHES_LOCS.append(Pos(x - __PCB_ORIGIN[0], -(y - __PCB_ORIGIN[1]) - row * UNIT))
for x, y, r, _, _ in __SWITCHES_THUMBS_LOCS_RAW:
    SWITCHES_LOCS.append(Pos(x - __PCB_ORIGIN[0], -(y - __PCB_ORIGIN[1])) * Rot(0, 0, r))

PLATE_HOLES_LOCS = [Pos(x, -y, 0) for x, y in [
    np.array([29.6075, 45.28]) - __PCB_ORIGIN,
    np.array([29.6075, 64.28]) - __PCB_ORIGIN,
    np.array([105.5755, 41.7655]) - __PCB_ORIGIN,
    np.array([119.3305, 89.1350]) - __PCB_ORIGIN
]]

PCB_BACKLEDS_LOCS = [Pos(x, -y, 0) for x, y in [
    np.array([39.1075, 45.28]) - __PCB_ORIGIN,
    np.array([39.1075, 64.28]) - __PCB_ORIGIN,
    np.array([77.1075, 38.155]) - __PCB_ORIGIN,
    np.array([77.1075, 76.155]) - __PCB_ORIGIN,
    np.array([115.1075, 42.905]) - __PCB_ORIGIN,
    np.array([115.1075, 80.905]) - __PCB_ORIGIN
]]

PCB_USB_LOC = Pos(
    134.8139 - __PCB_ORIGIN[0],
    -(26.7694 - __PCB_ORIGIN[1]),
    3.16 / 2
) * Rot(-90, 0, 0)

PCB_TRRS_LOC = Pos(
    142.6345 + 3.6 - __PCB_ORIGIN[0],
    -(74.292 - __PCB_ORIGIN[1]),
    2.5
) * Rot(0, 90, 0)

PCB_TRRS_OUTLINE = Pos(
    142.6345 + 3.6 - __PCB_ORIGIN[0],
    -(74.292 - __PCB_ORIGIN[1]),
    0
) * (
    Rectangle(2, 5, align=(Align.MAX, Align.CENTER))
    + Pos(-2, 0, 0) * Rectangle(12.2, 6.1, align=(Align.MAX, Align.CENTER))
)

PCB_PROMICRO_OUTLINE = Pos(
    134.8139 -__PCB_ORIGIN[0],
    -(61.1700 - __PCB_ORIGIN[1]),
    0
) * (
    Rectangle(18.5, 34.92, align=(Align.CENTER, Align.MIN))
    + Rectangle(8.92, 36.60, align=(Align.CENTER, Align.MIN))
)

In [ ]:
# 如有需要，导出按键外边框以协助后续外形设计

# SWITCHES_BOUNDARY = Sketch() + [loc * Rectangle(*size) for loc, size in zip(SWITCHES_LOCS, SWITCHES_SIZES)]
# exporter = ExportSVG()
# exporter.add_layer('switches').add_shape(SWITCHES_BOUNDARY)
# exporter.write('switches.svg')

## 外形设计

In [ ]:
def import_svg(file_path: str):
    code, val = import_svg_as_buildline_code(file_path)
    exec(code)
    return locals()[val].line

In [ ]:
LARGE_ENOUGH = 100. # “足够大”的值，用于减集穿过所有

In [ ]:
# 轴体参数

# 轴在定位板顶面之下的部分。用于在定位板上挖孔
SWITCH_BELOW_WIDTH, SWITCH_BELOW_DEPTH = (14.1, 5.)
SWITCH_CLICK_SPACE, SWITCH_CLICK_DIAMETER = (1.41, 15.)
SWITCH_CLICK_WIDTH, SWITCH_CLICK_WIDTH, SWITCH_CLICK_HEIGHT = (4., 3., SWITCH_BELOW_DEPTH - SWITCH_CLICK_SPACE)

# 轴在定位板顶面之上的部分。用于在外壳上为键帽留出空间
SWITCH_ABOVE_WIDTH, SWITCH_ABOVE_HEIGHT = (UNIT, 20.)

In [ ]:
# 定位板参数

PLATE_OFFSET = 10 # 定位板最低处的高度
PLATE_TILTING = 12.5 # 定位板的倾斜度

PLATE_THICKNESS = 5 # 定位板的厚度。5mm即为最大厚度，此时定位板与pcb直接接触

CASE_PLATE_PLANE = Plane( # 定位板所在平面。该平面为最重要的平面
    Pos(0, 0, PLATE_OFFSET) * Rot(0, -PLATE_TILTING, 0) * Rectangle(1, 1).face()
)

In [60]:
# 整体外形参数

CASE_SHELL_THICKNESS = 4 # 外壳厚度
CASE_TOP_OFFSET = 8 # 外壳顶面相对定位板的高度，也即键轴周围护栏的高度
CASE_TOP_CHAMFER = (1.5, 2.) # 外壳顶面四周的倒角尺寸
CASE_BOTTOM_THICKNESS = 2.

CASE_TOP_PLANE = Plane(CASE_PLATE_PLANE * Rectangle(1, 1).face().offset(CASE_TOP_OFFSET)) # 外壳顶面所在平面

CASE_TOP = make_face(import_svg('design/top.svg'))
CASE_BOTTOM = make_face(import_svg('design/bottom.svg'))
CASE_POCKET = make_face(import_svg('design/pocket.svg'))

In [ ]:
# PCB和接口参数

PCB_THICKNESS = 1.6 # PCB板厚度，需要根据实际PCB厚度调整

CONNECTOR_THICKNESS = 1.85 # 凯华热插拔轴座厚度

USB_HOLE_WIDTH, USB_HOLE_HEIGHT = 13., 7. # USB插槽尺寸

TRRS_DIAMETER = 8 # 耳机线接口直径
TRRS_EXTRA_POCKET_OFFSET = 4 # 耳机额外接口槽相对接口本身的偏移。由于垂直的3.5mm耳机接口与外壳会发生冲突，故特开此槽
TRRS_EXTRA_POCKET_WIDTH = 20 # 耳机额外接口槽宽度。
TRRS_EXTRA_POCKET_TAPER = 45 # 耳机额外接口槽斜度

PCB_PLANE = Plane(CASE_PLATE_PLANE * Rectangle(1, 1).face().offset(-5))

USB_LOC = PCB_PLANE * PCB_USB_LOC
TRRS_LOC = PCB_PLANE * PCB_TRRS_LOC

PCB_OUTLINE = make_face(import_svg('design/pcb.svg'))
OLED_POCKET = make_face(import_svg('design/oled.svg'))
PROMICRO_POCKET = offset(PCB_PROMICRO_OUTLINE, amount=0.25, kind=Kind.INTERSECTION)
TRRS_POCKET = offset(PCB_TRRS_OUTLINE, amount=0.25, kind=Kind.INTERSECTION)

PCB_POCKET_OFFSET = 12. # 保证PCB可以正常放入的最小开槽深度

In [ ]:
# PCB和定位板的固定相关参数

# 选用M2*6铜柱
PLATE_SCREW_DIAMETER = 2 # 螺丝直径
PLATE_COUNTERSUNK_TAPER = 90 / 2 # 沉头角度
PLATE_COUNTERSUNK_DEPTH = 1.3 # 沉头深度
PLATE_COUNTERSUNK_DIAMETER = 3.8 # 沉头直径
PLATE_SPACER_HEIGHT = 6 # 铜柱高度
PLATE_SPACER_HOLE_DIAMETER = 4 # 在定位板上为铜柱开的孔的直径，需要比铜柱外径大
PLATE_SPACER_HOLE_DEPTH = ( # 在定位板上为铜柱开的孔的深度，为了使背板能刚好把PCB夹住
    PLATE_SPACER_HEIGHT - PCB_THICKNESS - CONNECTOR_THICKNESS - 0.5
) 
BACKPLATE_THICKNESS = 2.

# 根据定位孔的位置和背光LED的位置，计算能挡住上述内容的最小背板形状
__BACK_PLATE_ANCHORS = [
    (loc.position.X, loc.position.Y)
    for loc in [*PLATE_HOLES_LOCS, *PCB_BACKLEDS_LOCS]
]
from scipy.spatial import ConvexHull
BACK_PLATE = offset(make_face(Polyline(*[
    __BACK_PLATE_ANCHORS[idx]
    for idx in ConvexHull(__BACK_PLATE_ANCHORS).vertices
], close=True)), 10)

In [ ]:
# 底座参数。该系列参数将同时用于腕托的底座

# 选用M3滚花螺母
FEET_OUTLINE_DIAMETER = 8 # 底座连接孔台面的外径
FEET_ENCHASE_HOLE_DIAMETER = 4. # 底座镶嵌螺母的孔的外径。M3滚花螺母外径为4.2，此处略小
FEET_ENCHASE_HOLE_DEPTH = 3 * 2 # 镶嵌螺母的孔的深度。M3滚花螺母高度为3，此处留出余量，避免镶嵌时树脂怼到螺纹里

FEET_SCREW_DIAMETER = 3 # 螺丝直径
FEET_COUNTERSUNK_TAPER = 90 / 2 # 沉头角度
FEET_COUNTERSUNK_DEPTH = 1.7 # 沉头深度
FEET_COUNTERSUNK_DIAMETER = 5.5 # 沉头直径

__CASE_BOTTOM_INLINE = offset(CASE_BOTTOM, -max(CASE_SHELL_THICKNESS, FEET_OUTLINE_DIAMETER / 2))
CASE_FEET_LOCS = [ # 键盘外壳的底座连接孔位置
    Pos(vertex.X, vertex.Y, vertex.Z)
    for vertex in (__CASE_BOTTOM_INLINE.edges().sort_by(Axis.X)[0].vertices()
                   + __CASE_BOTTOM_INLINE.edges().sort_by(Axis.X)[-1].vertices())
]

In [ ]:
# 腕托参数

# 石腕托形状参数
WRIST_REST = make_face(fillet(import_svg('design/wrist_rest.svg').vertices(), 8))
WRIST_REST_TILTING = math.atan2(6, 40) * 180. / math.pi # 根据腕托形状计算腕托倾斜角
WRIST_REST_THICKNESS1, WRIST_REST_THICKNESS2 = 12., 6.

# 以腕托中心为原点的平面
WRIST_TOP_PLANE = Plane(
    CASE_TOP_PLANE * Pos(0, 0, -CASE_TOP_CHAMFER[1])
    * Pos(48.1683, -123.4488, 0) * Rot(0, 0, -13.5578)
    * Rectangle(1, 1).face()
)

# 腕托架形状参数
WRIST_REST_POCKET_DEPTH = 10.
WRIST_PLATE_SHELL_THICKNESS = 3.
WRIST_PLATE_BOTTOM = make_face(import_svg('design/wrist_bottom.svg'))
WRIST_PLATE_TOP = make_face(import_svg('design/wrist_top.svg'))

__WRIST_PLATE_BOTTOM_VERTICES = sorted(
    offset(WRIST_PLATE_BOTTOM, -max(WRIST_PLATE_SHELL_THICKNESS, FEET_OUTLINE_DIAMETER / 2)).vertices(),
    key=lambda v: (Vector(v) - (48.1683, -123.4488, 0)).length, reverse=True
)
WRIST_PLATE_FEET_LOCS = [ # 腕托架的底座连接孔位置
    Pos(vertex.X, vertex.Y, vertex.Z)
    for vertex in __WRIST_PLATE_BOTTOM_VERTICES[0:4]
]

In [ ]:
# 外壳与腕托连接的磁铁的参数

MAGNET_DIAMETER, MAGNET_THICKNESS = 8., 2. # 磁铁形状
MAGNET_HOLE_DIAMETER = MAGNET_DIAMETER + 0.5 # 磁铁安装孔直径，留有胶水余量
MAGNET_HOLE_DEPTH = MAGNET_THICKNESS + 0.5 # 朝外的磁铁安装孔深度，留有余量，防止磁铁撞碎
MAGNET_SHELL_THICKNESS = 0.6 # 朝内的磁铁安装孔距离外侧面的厚度，需要考虑材料最小壁厚

## 生成外壳

In [61]:
# 分上下两部分分别生成外壳实体

# 放样得到基础外形
__case_solid = loft([
    CASE_BOTTOM,
    CASE_TOP_PLANE * Pos(0, 0, -CASE_TOP_CHAMFER[1]) * CASE_TOP
], ruled=True)

# 形状向内偏移，得到外壳
__case_shell = offset(__case_solid, -CASE_SHELL_THICKNESS, openings=__case_solid.faces().sort_by(Axis.Z)[0])

# 添加顶面的倒角
__case_solid += loft([
    CASE_TOP_PLANE * Pos(0, 0, -CASE_TOP_CHAMFER[1]) * CASE_TOP,
    CASE_TOP_PLANE * offset(CASE_TOP, -CASE_TOP_CHAMFER[0], kind=Kind.INTERSECTION)
])

# 由基础外形的上半部分和外壳的下半部分组成外壳实体
case_: Part = split(__case_solid, PCB_PLANE, keep=Keep.TOP) + split(__case_shell, PCB_PLANE, keep=Keep.BOTTOM)

# 减去开槽，形成按键围栏
case_ -= extrude(CASE_TOP_PLANE * CASE_POCKET, -CASE_TOP_OFFSET)

# 减去PCB区域，保证PCB能正常安装
case_ -= extrude(PCB_PLANE * PCB_OUTLINE, -LARGE_ENOUGH)
case_ -= extrude(PCB_PLANE * PROMICRO_POCKET, -PCB_POCKET_OFFSET)
case_ -= extrude(PCB_PLANE * TRRS_POCKET, -PCB_POCKET_OFFSET)

# 减去定位板按键孔位
__kailh_pocket = Part() + (
    Plane.YX * Box(SWITCH_BELOW_WIDTH, SWITCH_BELOW_WIDTH, SWITCH_BELOW_DEPTH, align=(Align.CENTER, Align.CENTER, Align.MIN))
    + Plane.YX * Pos(0, 0, SWITCH_CLICK_SPACE) * Box(SWITCH_CLICK_DIAMETER, SWITCH_CLICK_WIDTH,
                                                     SWITCH_CLICK_HEIGHT, align=(Align.CENTER, Align.CENTER, Align.MIN))
    + Plane.XY * Box(SWITCH_ABOVE_WIDTH, SWITCH_ABOVE_WIDTH, SWITCH_ABOVE_HEIGHT, align=(Align.CENTER, Align.CENTER, Align.MIN))
)
case_ -= [
    CASE_PLATE_PLANE * loc * __kailh_pocket
    for loc in SWITCHES_LOCS
]

# 生成PCB定位孔
case_ -= [
    PCB_PLANE * loc * extrude(Circle(PLATE_SPACER_HOLE_DIAMETER / 2), amount=PLATE_SPACER_HOLE_DEPTH)
    for loc in PLATE_HOLES_LOCS
]  # M2铜柱沉孔
case_ -= [
    CASE_PLATE_PLANE * loc * (extrude(Circle(PLATE_SCREW_DIAMETER / 2), amount=-PLATE_THICKNESS)
                              + extrude(Circle(PLATE_COUNTERSUNK_DIAMETER / 2),
                                        amount=-PLATE_COUNTERSUNK_DEPTH, taper=PLATE_COUNTERSUNK_TAPER))
    for loc in PLATE_HOLES_LOCS
]  # M2沉头螺丝孔

# 减去主控、OLED、TRRS
case_ -= extrude(CASE_PLATE_PLANE * OLED_POCKET, -PLATE_THICKNESS)
case_ -= extrude(CASE_PLATE_PLANE * PROMICRO_POCKET, -PLATE_THICKNESS)
case_ -= extrude(CASE_PLATE_PLANE * TRRS_POCKET, -PLATE_THICKNESS)

# 减去接口
# 考虑到接口均超出了PCB，此处向内偏移一段距离以同时削减出留给元件的空间
# 按理说这个应该放在PCB_OUTLINE中的，但是懒
case_ -= USB_LOC * Pos(0, 0, -3) * extrude(Rectangle(USB_HOLE_WIDTH, USB_HOLE_HEIGHT), amount=LARGE_ENOUGH)
case_ -= TRRS_LOC * Pos(0, 0, -3) * (
    Cylinder(TRRS_DIAMETER / 2, LARGE_ENOUGH, align=[Align.CENTER, Align.CENTER, Align.MIN])
    + Pos(0, 0, TRRS_EXTRA_POCKET_OFFSET + 3) * extrude(
        Circle(TRRS_DIAMETER / 2) + Rectangle(TRRS_DIAMETER, TRRS_EXTRA_POCKET_WIDTH, align=(Align.CENTER, Align.MIN)),
        amount=LARGE_ENOUGH, taper=-TRRS_EXTRA_POCKET_TAPER
    )
)

# 生成底板连接孔
case_ += [  # 生成孔位所在圆台
    extrude(loc * Circle(FEET_OUTLINE_DIAMETER / 2), until=Until.NEXT, target=case_)
    for loc in CASE_FEET_LOCS
]
case_ -= [  # 挖孔
    loc * Cylinder(FEET_ENCHASE_HOLE_DIAMETER / 2, FEET_ENCHASE_HOLE_DEPTH * 2)
    for loc in CASE_FEET_LOCS
]

# 生成腕托磁铁槽
__wrist_rest_mag_faces = (  # 选择朝前的四个面中中间两个
    split(
        loft([
            CASE_BOTTOM,
            CASE_TOP_PLANE * Pos(0, 0, -CASE_TOP_CHAMFER[1]) * CASE_TOP
        ], ruled=True),
        PCB_PLANE, keep=Keep.BOTTOM
    ).faces()
    .filter_by_position(Axis.Y, -math.inf, -57).filter_by_position(Axis.X, 19, 95)  # 此处筛选条件很难参数化
)
wrist_rest_mag_planes = [  # 生成接触平面
    Plane(face.intersect(Axis(face.center(), face.normal_at())), z_dir=face.normal_at())
    for face in __wrist_rest_mag_faces
]
case_ -= [  # 向内减去磁铁开槽
    extrude(
        plane.offset(-MAGNET_SHELL_THICKNESS) * Circle(MAGNET_HOLE_DIAMETER / 2),
        amount=-CASE_SHELL_THICKNESS
    )
    for plane in wrist_rest_mag_planes
]

# 完成建模
case_.label = 'case'

In [62]:
if VISUALIZE:
    show(case_)

+

In [ ]:
if EXPORT:
    case_.export_step('build/case.step')

## 生成PCB背板

In [ ]:
backplate: Part = extrude(BACK_PLATE, -BACKPLATE_THICKNESS) - [
    loc * Cylinder(PLATE_SCREW_DIAMETER / 2, 6)
    for loc in PLATE_HOLES_LOCS
]

backplate.label = 'pcb back'

In [ ]:
# 生成用于加工的dxf
backplate_sketch: Sketch = BACK_PLATE - [
    loc * Circle(PLATE_SCREW_DIAMETER / 2)
    for loc in PLATE_HOLES_LOCS
]

In [ ]:
if EXPORT:
    backplate.export_step('build/back.step')
    
    dxf_exporter = ExportDXF()
    dxf_exporter.add_layer('backplate')
    dxf_exporter.add_shape(backplate_sketch)
    dxf_exporter.write('build/back.dxf')

## 生成底板

In [ ]:
bottom: Part = extrude(CASE_BOTTOM, -CASE_BOTTOM_THICKNESS) - [
    loc * (
        Cylinder(FEET_SCREW_DIAMETER / 2, LARGE_ENOUGH)
        + Pos(0, 0, -CASE_BOTTOM_THICKNESS) *
        extrude(Circle(FEET_COUNTERSUNK_DIAMETER / 2),
                amount=FEET_COUNTERSUNK_DEPTH, taper=FEET_COUNTERSUNK_TAPER)
    )
    for loc in CASE_FEET_LOCS
]

bottom.label = 'bottom'

In [ ]:
# 生成用于加工的dxf
bottom_sketch: Sketch = CASE_BOTTOM - [
    loc * Circle(FEET_SCREW_DIAMETER / 2)
    for loc in CASE_FEET_LOCS
]

In [ ]:
if EXPORT:
    bottom.export_step('build/bottom.step')

    dxf_exporter = ExportDXF()
    dxf_exporter.add_layer('backplate')
    dxf_exporter.add_shape(bottom_sketch)
    dxf_exporter.write('build/bottom.dxf')

## 生成腕托

In [ ]:
# 腕托托架

# 放样得到基础外形
__solid = loft([
    WRIST_PLATE_BOTTOM,
    CASE_TOP_PLANE * Pos(0, 0, -CASE_TOP_CHAMFER[1]) * WRIST_PLATE_TOP
], ruled=True)

# 上半部分用于容纳腕托
__above = split(__solid, WRIST_TOP_PLANE.offset(-(WRIST_REST_POCKET_DEPTH + WRIST_PLATE_SHELL_THICKNESS)), keep=Keep.TOP)

# 下半部分生成外壳
__shell = offset(
    __solid, amount=-WRIST_PLATE_SHELL_THICKNESS,
    openings=__solid.faces().sort_by(Axis(WRIST_TOP_PLANE.origin, WRIST_TOP_PLANE.z_dir))[0]
)
__below = split(__shell, WRIST_TOP_PLANE.offset(-(10 + WRIST_PLATE_SHELL_THICKNESS)), keep=Keep.BOTTOM)

# 拼接两部分
wrist_rest_plate = __above + __below

# 生成底板连接孔
wrist_rest_plate += __solid & [
    extrude(loc * Circle(FEET_OUTLINE_DIAMETER / 2), amount=LARGE_ENOUGH)
    for loc in WRIST_PLATE_FEET_LOCS
]  # 生成孔位所在圆台
wrist_rest_plate -= [
    loc * Cylinder(FEET_ENCHASE_HOLE_DIAMETER / 2, FEET_ENCHASE_HOLE_DEPTH * 2)
    for loc in WRIST_PLATE_FEET_LOCS
]  # 挖孔

# 切割出斜面
wrist_rest_plate = split(
    wrist_rest_plate,
    Plane(WRIST_TOP_PLANE * Rot(WRIST_REST_TILTING, 0, 0) * Rectangle(1, 1).face()),
    keep=Keep.BOTTOM
)

# 减去腕托槽
wrist_rest_plate -= WRIST_TOP_PLANE * extrude(WRIST_REST, amount=-WRIST_REST_POCKET_DEPTH)

# 挖磁铁槽
wrist_rest_plate -= [
    plane * Cylinder(MAGNET_HOLE_DIAMETER / 2, 2 * MAGNET_HOLE_DEPTH) # 向外挖槽。使用双向距离保证向外连通
    for plane in wrist_rest_mag_planes
]

wrist_rest_plate.label = 'wrist rest plate'

In [ ]:
# 腕托本体

wrist_rest: Part = split(
    extrude(WRIST_REST, amount=12),
    Plane(Pos(0, 0, WRIST_REST_THICKNESS1) * Rot(WRIST_REST_TILTING, 0, 0) * Rectangle(1, 1).face()),
    keep=Keep.BOTTOM
)

wrist_rest = fillet(
    wrist_rest.edges().filter_by(Axis.Z, reverse=True).filter_by_position(Axis.Z, 2, 20)
    - wrist_rest.edges().filter_by_position(Axis.Y, -10, 20),
    radius=1
)

wrist_rest.label = 'wrist rest'

In [ ]:
if VISUALIZE:
    show(wrist_rest_plate, WRIST_TOP_PLANE * Pos(0, 0, -10) * wrist_rest, case_)

In [ ]:
if EXPORT:
    wrist_rest_plate.export_step('build/wrist_rest.step')

## 生成腕托底板

In [ ]:
wrist_rest_bottom: Part = extrude(WRIST_PLATE_BOTTOM, -CASE_BOTTOM_THICKNESS) - [
    loc * (
        Cylinder(FEET_SCREW_DIAMETER / 2, LARGE_ENOUGH)
        + Pos(0, 0, -CASE_BOTTOM_THICKNESS) *
        extrude(Circle(FEET_COUNTERSUNK_DIAMETER / 2),
                amount=FEET_COUNTERSUNK_DEPTH, taper=FEET_COUNTERSUNK_TAPER)
    )
    for loc in WRIST_PLATE_FEET_LOCS
]
wrist_rest_bottom.label = 'bottom'

In [ ]:
# 生成用于加工的dxf

wrist_rest_bottom_sketch: Sketch = WRIST_PLATE_BOTTOM - [
    loc * Circle(FEET_SCREW_DIAMETER / 2)
    for loc in WRIST_PLATE_FEET_LOCS
]

In [ ]:
if EXPORT:
    wrist_rest_bottom.export_step('build/wrist_rest_bottom.step')

    dxf_exporter = ExportDXF()
    dxf_exporter.add_layer('wrist_rest_bottom_sketch')
    dxf_exporter.add_shape(wrist_rest_bottom_sketch)
    dxf_exporter.write('build/wrist_rest_bottom.dxf')

## 键盘假组

In [ ]:
ass_case = Compound(children=[
    case_, bottom,
    PCB_PLANE.offset(-(- PLATE_SPACER_HOLE_DEPTH + PLATE_SPACER_HEIGHT)) * backplate
], label='case')

ass_wrist_rest = Compound(children=[
    wrist_rest_plate, wrist_rest_bottom, WRIST_TOP_PLANE * Pos(0, 0, -10) * wrist_rest
], label='wrist rest')

assembly = Compound(children=[
    ass_case,
    ass_wrist_rest
], label='crkbd case')

In [ ]:
if VISUALIZE:
    show(assembly)

In [ ]:
if EXPORT:
    assembly.export_step('build/assembly.step')